In [1]:
import numpy as np

from sklearn.metrics import mean_squared_error

In [2]:
result_dir = '../results/'

# TablePrinter class

In [3]:
class TablePrinter:
    
    def __init__(self, header='', metric_fns=[], precisions=[]):
        
        # Functions for metrics
        self.metric_fns = metric_fns
        self.n_metrics = len(self.metric_fns)
        self.precisions = precisions if len(precisions) == self.n_metrics else [2] * self.n_metrics
        
        # Total table and current block
        self.table = ''
        self.block_entries = []

        # Formatting options
        self.indent = '   '
        self.col_sep = ' & '
        self.end_row = r'\\'
        self.midrule = r'\midrule'
        self.end_line = '\n'
        self.emphasis_begin = r'\mathbf{'
        self.emphasis_end = r'}'
        
        # Header
        self.table = self.indent + header + self.end_row + self.end_line
    
    
    def finalise_block(self):

        # Find best performance
        block_metrics = [line[2:] for line in self.block_entries]
        block_metrics = np.array(block_metrics)
        block_best = []
        for i in range(self.n_metrics):
            try:
                block_best.append(np.nanargmin(block_metrics[:,i]))
            except ValueError:
                block_best.append(-1)

        # Format entries
        text = ''
        for i, line in enumerate(self.block_entries):
            
            # Labels
            text += self.indent + line[0] + self.col_sep + line[1] + self.col_sep
            
            # Metrics
            for j in range(self.n_metrics):
                if np.isfinite(line[j + 2]):
                    text += self.format_number(line[j + 2], self.precisions[j], emphasize=(i == block_best[j]))
                if j == len(line) - 3:
                    text += self.end_row + self.end_line
                else:
                    text += self.col_sep

        # Add to document and reset for next block
        self.table += text
        self.block_entries = []
    
    
    def new_block(self):
        self.finalise_block()
        self.table += self.indent + self.midrule + self.end_line
    
    
    def format_number(self,
                      number,
                      precision=2,
                      trailing_zeros=True,
                      fix_minus_zero=True,
                      latex_math_mode=True,
                      emphasize=False):
        if precision == 0:
            temp =  str(int(round(number,precision)))
        elif trailing_zeros:
            temp =  ('{:.' + str(precision) + 'f}').format(round(number,precision))
        else:
            temp =  str(round(number,precision))
        if fix_minus_zero and len(temp) > 0:
            if temp[0] == '-' and float(temp) == 0.:
                temp = temp[1:]
        if latex_math_mode:
            if emphasize:
                temp = '$\mathbf{' + temp + '}$'
            else:
                temp = '$' + temp + '$'
        elif emphasize:
            temp = r'\emph{' + temp + r'}'
        return temp
    
    
    def add(self, col1, col2, filename, folder='parameterized'):
        
        # Label columns
        line = [col1, col2]
        
        # Metrics
        for fn in self.metric_fns:
            try:
                value = fn(filename, folder)
            except (IOError, ValueError):
                value = np.nan
            line.append(value)

        self.block_entries.append(line)
    
    
    def print(self):
        self.finalise_block()
        return self.table

# Metrics

In [4]:
def mse_log_r_nottrained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_nottrained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_nottrained_' + filename + '.npy'))
    return mean_squared_error(log_r_truth, log_r_estimated)

def mse_log_r_trained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_trained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_trained_' + filename + '.npy'))
    return mean_squared_error(log_r_truth, log_r_estimated)

def mse_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')
    return mean_squared_error(expected_log_r_truth, expected_log_r_estimated)

def mse_delta_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')
    expected_log_r_truth -= np.min(expected_log_r_truth)
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')
    expected_log_r_estimated -= np.min(expected_log_r_estimated)
    return mean_squared_error(expected_log_r_truth, expected_log_r_estimated)

def mse_score_nottrained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_nottrained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_nottrained_' + filename + '.npy')
    return mean_squared_error(t_truth, t_estimated)

def mse_score_trained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_trained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_trained_' + filename + '.npy')
    return mean_squared_error(t_truth, t_estimated)

# Tables

In [12]:
labels = ['PbP carl (raw)',
          'PbP carl (cal.)',
          'Param carl (raw)',
          'Param carl (cal.)',
          'Param score (raw)',
          'Param score (cal.)',
          'Param carl + score (raw)',
          'Param carl + score (cal.)',
          'PbP regression (raw)',
          'Param regression (raw)',
          'Param regr.\ + score (raw)']

point_by_point = [True, True,
                 False, False, False, False, False, False,
                 True,
                 False, False]

filenames = ['carl',
             'carl_calibrated',
             'carl',
             'carl_calibrated',
             'score',
             'score_calibrated',
             'combined',
             'combined_calibrated',
             'regression',
             'regression',
             'combinedregression']

In [16]:
table1 = TablePrinter(r'Algorithm & Setup'
             + r' & $\MSE_\boldx \left[ \log r(\boldx; \boldtheta_A, \boldtheta_1) \right]$'
             + r' & $\MSE_\boldx \left[ \log r(\boldx; \boldtheta_B, \boldtheta_1) \right]$'
             + r' & $\MSE_\boldtheta \left[ E[\log r(\boldx; \boldtheta, \boldtheta_1)] \right]$'
             + r' & $\MSE_\boldtheta \left[ E[\log r(\boldx; \boldtheta, \hat{\boldtheta})] \right]$',
            [mse_log_r_trained, mse_log_r_nottrained, mse_expected_log_r, mse_delta_expected_log_r],
                     precisions=[3,3,2,2])

for i, (label, filename, pbp) in enumerate(zip(labels[:4], filenames[:4], point_by_point[:4])):
    
    if i > 0:
        table1.new_block()
        
    if pbp:
        table1.add(label, 'PbP (1)', filename + '_shallow', 'point_by_point')
        table1.add('', 'PbP (2)', filename, 'point_by_point')
        table1.add('', 'PbP (3)', filename + '_deep', 'point_by_point')
        table1.add('', r'PbP learning $\log r(\boldx)$ (1)', filename + '_learnlogr_shallow', 'point_by_point')
        table1.add('', r'PbP learning $\log r(\boldx)$ (2)', filename + '_learnlogr', 'point_by_point')
        table1.add('', r'PbP learning $\log r(\boldx)$ (3)', filename + '_learnlogr_deep', 'point_by_point')
    
    else:
        table1.add(label, 'Baseline (1)', filename + '_shallow')
        table1.add('', 'Baseline (2)', filename)
        table1.add('', 'Baseline (3)', filename + '_deep')

        table1.add('', r'Random $\boldtheta$ (1)', filename + '_random_shallow')
        table1.add('', r'Random $\boldtheta$ (2)', filename + '_random')
        table1.add('', r'Random $\boldtheta$ (3)', filename + '_random_deep')

        table1.add('', 'Physics-aware (1)', filename + '_aware_shallow')
        table1.add('', 'Physics-aware (2)', filename + '_aware')
        table1.add('', 'Physics-aware (3)', filename + '_aware_deep')

print(table1.print())

   Algorithm & Setup & $\MSE_\boldx \left[ \log r(\boldx; \boldtheta_A, \boldtheta_1) \right]$ & $\MSE_\boldx \left[ \log r(\boldx; \boldtheta_B, \boldtheta_1) \right]$ & $\MSE_\boldtheta \left[ E[\log r(\boldx; \boldtheta, \boldtheta_1)] \right]$ & $\MSE_\boldtheta \left[ E[\log r(\boldx; \boldtheta, \hat{\boldtheta})] \right]$\\
   PbP carl (raw) & PbP (1) & $0.206$ & $0.229$ & $50.54$ & $5.98$\\
    & PbP (2) & $0.177$ & $0.203$ & $48.99$ & $4.75$\\
    & PbP (3) & $0.201$ & $0.272$ & $49.31$ & $5.97$\\
    & PbP learning $\log r(\boldx)$ (1) & $0.189$ & $0.193$ & $49.15$ & $\mathbf{4.14}$\\
    & PbP learning $\log r(\boldx)$ (2) & $0.269$ & $0.196$ & $48.51$ & $4.59$\\
    & PbP learning $\log r(\boldx)$ (3) & $\mathbf{0.114}$ & $\mathbf{0.157}$ & $\mathbf{47.81}$ & $4.80$\\
   \midrule
   PbP carl (cal.) & PbP (1) & $0.094$ & $0.121$ & $3.91$ & $\mathbf{3.82}$\\
    & PbP (2) & $0.139$ & $0.239$ & $3.91$ & $3.87$\\
    & PbP (3) & $0.099$ & $0.132$ & $3.95$ & $3.90$\\
    & PbP l

In [14]:
table2 = TablePrinter(r'Algorithm & Setup'
             + r' & $\MSE_\boldx \left[ \log r(\boldx; \boldtheta_A, \boldtheta_1) \right]$'
             + r' & $\MSE_\boldx \left[ \log r(\boldx; \boldtheta_B, \boldtheta_1) \right]$'
             + r' & $\MSE_\boldtheta \left[ E[\log r(\boldx; \boldtheta, \boldtheta_1)] \right]$'
             + r' & $\MSE_\boldtheta \left[ E[\log r(\boldx; \boldtheta, \hat{\boldtheta})] \right]$',
            [mse_log_r_trained, mse_log_r_nottrained, mse_expected_log_r, mse_delta_expected_log_r],
                     precisions=[3,3,2,2])

for i, (label, filename, pbp) in enumerate(zip(labels[4:8], filenames[4:8], point_by_point[4:8])):
    
    if i > 0:
        table2.new_block()
        
    if pbp:
        table2.add(label, 'PbP (1)', filename + '_shallow', 'point_by_point')
        table2.add('', 'PbP (2)', filename, 'point_by_point')
        table2.add('', 'PbP (3)', filename + '_deep', 'point_by_point')
        table2.add('', 'PbP learning log r (1)', filename + '_learnlogr_shallow', 'point_by_point')
        table2.add('', 'PbP learning log r (2)', filename + '_learnlogr', 'point_by_point')
        table2.add('', 'PbP learning log r (3)', filename + '_learnlogr_deep', 'point_by_point')
    
    else:
        table2.add(label, 'Baseline (1)', filename + '_shallow')
        table2.add('', 'Baseline (2)', filename)
        table2.add('', 'Baseline (3)', filename + '_deep')

        table2.add('', r'Random $\boldtheta$ (1)', filename + '_random_shallow')
        table2.add('', r'Random $\boldtheta$ (2)', filename + '_random')
        table2.add('', r'Random $\boldtheta$ (3)', filename + '_random_deep')

        table2.add('', 'Physics-aware (1)', filename + '_aware_shallow')
        table2.add('', 'Physics-aware (2)', filename + '_aware')
        table2.add('', 'Physics-aware (3)', filename + '_aware_deep')

print(table2.print())

   Algorithm & Setup & $\MSE_\boldx \left[ \log r(\boldx; \boldtheta_A, \boldtheta_1) \right]$ & $\MSE_\boldx \left[ \log r(\boldx; \boldtheta_B, \boldtheta_1) \right]$ & $\MSE_\boldtheta \left[ E[\log r(\boldx; \boldtheta, \boldtheta_1)] \right]$ & $\MSE_\boldtheta \left[ E[\log r(\boldx; \boldtheta, \hat{\boldtheta})] \right]$\\
   Param score (raw) & Baseline (1) & $\mathbf{0.060}$ & $0.067$ & $0.23$ & $0.25$\\
    & Baseline (2) & $0.060$ & $\mathbf{0.064}$ & $\mathbf{0.16}$ & $\mathbf{0.06}$\\
    & Baseline (3) & $0.104$ & $0.110$ & $15.30$ & $0.09$\\
    & Random $\boldtheta$ (1) & $0.099$ & $0.135$ & $3.51$ & $3.57$\\
    & Random $\boldtheta$ (2) & $0.082$ & $0.113$ & $3.76$ & $3.19$\\
    & Random $\boldtheta$ (3) & $0.135$ & $0.177$ & $14.09$ & $3.20$\\
    & Physics-aware (1) &  &  &  & \\
    & Physics-aware (2) &  &  &  & \\
    & Physics-aware (3) &  &  &  & \\
   \midrule
   Param score (cal.) & Baseline (1) & $0.100$ & $0.105$ & $0.96$ & $0.90$\\
    & Baseline (2) & $

In [15]:
table3 = TablePrinter(r'Algorithm & Setup'
             + r' & $\MSE_\boldx \left[ \log r(\boldx; \boldtheta_A, \boldtheta_1) \right]$'
             + r' & $\MSE_\boldx \left[ \log r(\boldx; \boldtheta_B, \boldtheta_1) \right]$'
             + r' & $\MSE_\boldtheta \left[ E[\log r(\boldx; \boldtheta, \boldtheta_1)] \right]$'
             + r' & $\MSE_\boldtheta \left[ E[\log r(\boldx; \boldtheta, \hat{\boldtheta})] \right]$',
            [mse_log_r_trained, mse_log_r_nottrained, mse_expected_log_r, mse_delta_expected_log_r],
                     precisions=[3,3,2,2])

for i, (label, filename, pbp) in enumerate(zip(labels[8:], filenames[8:], point_by_point[8:])):
    
    if i > 0:
        table3.new_block()
        
    if pbp:
        table3.add(label, 'PbP (1)', filename + '_shallow', 'point_by_point')
        table3.add('', 'PbP (2)', filename, 'point_by_point')
        table3.add('', 'PbP (3)', filename + '_deep', 'point_by_point')
    
    else:
        table3.add(label, 'Baseline (1)', filename + '_shallow')
        table3.add('', 'Baseline (2)', filename)
        table3.add('', 'Baseline (3)', filename + '_deep')

        table3.add('', r'Random $\boldtheta$ (1)', filename + '_random_shallow')
        table3.add('', r'Random $\boldtheta$ (2)', filename + '_random')
        table3.add('', r'Random $\boldtheta$ (3)', filename + '_random_deep')

        table3.add('', 'Physics-aware (1)', filename + '_aware_shallow')
        table3.add('', 'Physics-aware (2)', filename + '_aware')
        table3.add('', 'Physics-aware (3)', filename + '_aware_deep')

print(table3.print())

   Algorithm & Setup & $\MSE_\boldx \left[ \log r(\boldx; \boldtheta_A, \boldtheta_1) \right]$ & $\MSE_\boldx \left[ \log r(\boldx; \boldtheta_B, \boldtheta_1) \right]$ & $\MSE_\boldtheta \left[ E[\log r(\boldx; \boldtheta, \boldtheta_1)] \right]$ & $\MSE_\boldtheta \left[ E[\log r(\boldx; \boldtheta, \hat{\boldtheta})] \right]$\\
   PbP regression (raw) & PbP (1) & $0.011$ & $0.015$ & $3.79$ & $3.92$\\
    & PbP (2) & $0.005$ & $\mathbf{0.005}$ & $3.78$ & $\mathbf{3.83}$\\
    & PbP (3) & $\mathbf{0.004}$ & $0.006$ & $\mathbf{3.60}$ & $4.02$\\
   \midrule
   Param regression (raw) & Baseline (1) & $0.015$ & $0.022$ & $0.12$ & $0.12$\\
    & Baseline (2) & $\mathbf{0.004}$ & $\mathbf{0.005}$ & $0.16$ & $0.14$\\
    & Baseline (3) & $0.007$ & $0.007$ & $0.30$ & $0.29$\\
    & Random $\boldtheta$ (1) & $0.015$ & $0.021$ & $0.40$ & $0.22$\\
    & Random $\boldtheta$ (2) & $0.005$ & $0.007$ & $0.20$ & $\mathbf{0.02}$\\
    & Random $\boldtheta$ (3) & $0.007$ & $0.008$ & $\mathbf{0.04}$ & $

In [9]:
labels = ['Param carl',
          'Param score',
          'Param carl + score',
          'Param regression',
          'Param regr.\ + score']

filenames = ['carl',
             'score',
             'combined',
             'regression',
             'combinedregression']

table4 = TablePrinter(r'Algorithm & Setup'
             + r' & $\MSE_\boldx \left[ \log t(\boldx; \boldtheta_A, \boldtheta_1) \right]$'
             + r' & $\MSE_\boldx \left[ \log t(\boldx; \boldtheta_B, \boldtheta_1) \right]$',
            [mse_score_trained, mse_score_nottrained],
                     precisions=[2,2])

for i, (label, filename) in enumerate(zip(labels, filenames)):
    
    if i > 0:
        table4.new_block()
        
    table4.add(label, 'Baseline (1)', filename + '_shallow')
    table4.add('', 'Baseline (2)', filename)
    table4.add('', 'Baseline (3)', filename + '_deep')
    
    table4.add('', r'Random $\boldtheta$ (1)', filename + '_random_shallow')
    table4.add('', r'Random $\boldtheta$ (2)', filename + '_random')
    table4.add('', r'Random $\boldtheta$ (3)', filename + '_random_deep')

    table4.add('', 'Physics-aware (1)', filename + '_aware_shallow')
    table4.add('', 'Physics-aware (2)', filename + '_aware')
    table4.add('', 'Physics-aware (3)', filename + '_aware_deep')

print(table4.print())

   Algorithm & Setup & $\MSE_\boldx \left[ \log t(\boldx; \boldtheta_A, \boldtheta_1) \right]$ & $\MSE_\boldx \left[ \log t(\boldx; \boldtheta_B, \boldtheta_1) \right]$\\
   Param carl & Baseline (1) & $\mathbf{0.42}$ & $\mathbf{0.81}$\\
    & Baseline (2) & $1.12$ & $1.46$\\
    & Baseline (3) & $4.29$ & $2.85$\\
    & Random $\boldtheta$ (1) & $0.43$ & $0.85$\\
    & Random $\boldtheta$ (2) & $1.20$ & $1.45$\\
    & Random $\boldtheta$ (3) & $3.21$ & $3.31$\\
    & Physics-aware (1) &  & \\
    & Physics-aware (2) &  & \\
    & Physics-aware (3) &  & \\
   \midrule
   Param score & Baseline (1) & $0.19$ & $0.48$\\
    & Baseline (2) & $\mathbf{0.09}$ & $\mathbf{0.22}$\\
    & Baseline (3) & $0.10$ & $0.23$\\
    & Random $\boldtheta$ (1) & $0.40$ & $0.77$\\
    & Random $\boldtheta$ (2) & $0.37$ & $0.72$\\
    & Random $\boldtheta$ (3) & $0.40$ & $0.75$\\
    & Physics-aware (1) &  & \\
    & Physics-aware (2) &  & \\
    & Physics-aware (3) &  & \\
   \midrule
   Param carl + score 